### Prepare Dataset for training
- 1] Extract Frames from video
- 2] Label convertion (COCOformats - YOLO - VOC)
- 3] Display labels on images - 

In [1]:
import cv2
import os, glob
import json

In [2]:
def check_directory(path_to):
        """Check if the saving directory exist, create it if not"""
        if not os.path.isdir(path_to):
            os.makedirs(path_to)

        if path_to.endswith('/'):
            return(path_to)
        else:
            return(path_to + '/')

In [3]:
# Path to data
path_to_video_folder = '../data/train_videos/'
output_path = '../data/train_images/'
output_path = check_directory(output_path)

In [4]:
# List of videos
list_of_videos = os.listdir(path_to_video_folder)
print("{} video to be processed.".format(len(list_of_videos)))

25 video to be processed.


In [ ]:
# Loop to extract all the frame
for vid in list_of_videos:
    vid_name = vid.split('.')[0]
    vid_path = path_to_video_folder + vid
    vid_output_path = output_path + vid_name
    print("Processing video {} ...".format(vid_name))
    
    # Create sub-folder (maybe not needed?)
    vid_output_path = check_directory(vid_output_path)
    
    # Extract frames
    vid_cap = cv2.VideoCapture(vid_path)
    
    success, frame = vid_cap.read()
    frame_id = 0
    while success:
      cv2.imwrite(vid_output_path + str(frame_id) + '.png', frame)
      success, frame = vid_cap.read()
      frame_id += 1
    print("... extracted {} frames.".format(frame_id))

### Label generation
- coco: "bbox" : [x,y,width,height]
- yolo: "bbox" : [x, y,width,height] normalize over image size

COCO links:

- [coco_lib](https://github.com/waspinator/pycococreator/blob/master/pycococreatortools/pycococreatortools.py)
- [generator_example 1](https://github.com/waspinator/pycococreator/blob/master/examples/shapes/shapes_to_coco.py)
- [format explained](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch/#coco-dataset-format)
- [generator example 2](https://patrickwasp.com/create-your-own-coco-style-dataset/)
- [simple visualisation](https://github.com/waspinator/pycococreator/blob/master/examples/shapes/visualize_coco.ipynb)


In [5]:
# COCO dataset global
INFO = {
    "description": "AI on edge dataset",
    "url": "https://signate.jp/competitions/256/data",
    "version": "0.1",
    "year": 2020,
    "contributor": "MLT",
    "date_created": "2020/05"
}

LICENSES = [
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'Car',
        'supercategory': 'on_road',
    },
    {
        'id': 2,
        'name': 'Pedestrian',
        'supercategory': 'on_street',
    },
    {
        'id': 3,
        'name': 'Truck',
        'supercategory': 'on_road',
    },
    {
        'id': 4,
        'name': 'Signal',
        'supercategory': 'on_road',
    },
    {
        'id': 5,
        'name': 'Signs',
        'supercategory': 'on_road',
    },
    {
        'id': 6,
        'name': 'Bicycle',
        'supercategory': 'on_road',
    },
    {
        'id': 7,
        'name': 'Motorbike',
        'supercategory': 'on_road',
    },
    {
        'id': 8,
        'name': 'Bus',
        'supercategory': 'on_road',
    },
    {
        'id': 9,
        'name': 'Svehicle',
        'supercategory': 'on_road',
    },
    {
        'id': 10,
        'name': 'Train',
        'supercategory': 'on_road',
    },
]

In [6]:
# COCO field generation:
def generate_image_info(image_id, file_name):
    """Generate information related to image."""
    IMAGE_SIZE = (1936,1216)
    image_info = {
            "id": image_id,
            "file_name": file_name,
            "width": IMAGE_SIZE[0],
            "height": IMAGE_SIZE[1],
            "date_captured": "xx/xx/xx",
            "license": 0,
            "coco_url": "",
            "flickr_url": ""
    }

    return image_info

def generate_annotation_info(image_id, bbox, category_id, tracking_id):
    """Generate information related to annotation."""
    annot_info = {
            "segmentation": [],
            "area": 0,
            "iscrowd": 0,
            "image_id": image_id,
            "bbox": bbox,
            "category_id": category_id,
            "id": tracking_id
    }
    
    return annot_info

def to_coco_bbox(bbox):
    """Convert bbox_annot to coco format."""
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]
    
    return([bbox[0], bbox[2], width, height])

In [7]:
# Get path for annotation files (or we can try to do it from the .csv):
_path_for_data = "../data/"
path_coco_annot = check_directory(_path_for_data + 'COCO_annotations/')
train_annotations_path = os.path.join(_path_for_data, 'train_annotations')
train_annotations_files = os.listdir(train_annotations_path)

In [8]:
# Main loop
cat_dict = {'Car':1, 'Pedestrian':2,
            'Truck':3, 'Signal':4,
            'Signs':5, 'Bicycle':6,
            'Motorbike':7, 'Bus':8,
            'Svehicle':9, 'Train':10}

for train_annotations_file in train_annotations_files:
    
    coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }
    
    print("Generating COCO like dataset for: {}...".format(train_annotations_file))
    content_list = []
    with open(os.path.join(train_annotations_path, train_annotations_file)) as f:
        video_id = train_annotations_file.split('/')[-1].split('\\')[-1].split('.')[0].split('_')[1]

        # Load json
        annotation = json.load(f)
        frames = annotation["sequence"]
        frame_id = 0
        
        # Add COCO info for each frame: 
        for f in frames:
            # Add image:
            image_name = train_annotations_file.split('.')[0] + '/' + str(frame_id) + '.png'
            image_info = generate_image_info(frame_id, image_name)
            coco_output["images"].append(image_info)

            # Extract annotation information
            for cat in cat_dict:
                try:
                    for detected_cat in f[cat]:
                        coco_bbox  = to_coco_bbox(detected_cat['box2d'])
                        annot_info = generate_annotation_info(frame_id, coco_bbox, 
                                                              cat_dict[cat], detected_cat['id'])
                        coco_output["annotations"].append(annot_info)
                except Exception as e:
                    #print("No {} in this frame (or other error type)".format(e))
                    pass

            frame_id += 1
            
        # Write json file:
        json_file_name = _path_for_data + path_coco_annot + train_annotations_file.split('.')[0] + '.json'
        with open(json_file_name, 'w+') as output_json_file:
            json.dump(coco_output, output_json_file)
        
        output_json_file.close()
        print("...Done.")

Generating COCO like dataset for: train_09.json...
...Done.
Generating COCO like dataset for: train_01.json...
...Done.
Generating COCO like dataset for: train_10.json...
...Done.
Generating COCO like dataset for: train_13.json...
...Done.
Generating COCO like dataset for: train_06.json...
...Done.
Generating COCO like dataset for: train_14.json...
...Done.
Generating COCO like dataset for: train_23.json...
...Done.
Generating COCO like dataset for: train_12.json...
...Done.
Generating COCO like dataset for: train_02.json...
...Done.
Generating COCO like dataset for: train_24.json...
...Done.
Generating COCO like dataset for: train_00.json...
...Done.
Generating COCO like dataset for: train_21.json...
...Done.
Generating COCO like dataset for: train_07.json...
...Done.
Generating COCO like dataset for: train_16.json...
...Done.
Generating COCO like dataset for: train_03.json...
...Done.
Generating COCO like dataset for: train_05.json...
...Done.
Generating COCO like dataset for: train_

In [9]:
import coco_loader
import numpy as np
import IPython

path_coco_json = "../data/COCO_annotations/train_09.json"
path_to_image = '../data/train_images/'
coco_dataset = coco_loader.CocoDataset(path_coco_json, path_to_image)
coco_dataset.display_info()
coco_dataset.display_licenses()
coco_dataset.display_categories()

Dataset Info:
  description: AI on edge dataset
  url: https://signate.jp/competitions/256/data
  version: 0.1
  year: 2020
  contributor: MLT
  date_created: 2020/05

Licenses:
  id: 1
  name: Attribution-NonCommercial-ShareAlike License
  url: http://creativecommons.org/licenses/by-nc-sa/2.0/


Categories:
  super_category: on_road
    id 1: Car
    id 3: Truck
    id 4: Signal
    id 5: Signs
    id 6: Bicycle
    id 7: Motorbike
    id 8: Bus
    id 9: Svehicle
    id 10: Train

  super_category: on_street
    id 2: Pedestrian



In [ ]:
image_index = np.random.randint(0, 600)

In [ ]:
image_index = 0
html = coco_dataset.display_image(image_index, show_polys=False, show_crowds=False)
IPython.display.HTML(html)

In [ ]:
%matplotlib inline
from pycoco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

path_coco_json = "../data/COCO_annotations/train_09.json"
path_to_image = '../data/train_images/'
example_coco = COCO(path_coco_json)


In [ ]:
categories = example_coco.loadCats(example_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

category_names = set([category['supercategory'] for category in categories])
print('Custom COCO supercategories: \n{}'.format(' '.join(category_names)))

In [ ]:

category_ids = example_coco.getCatIds(catNms=['Car'])
image_ids = example_coco.getImgIds(catIds=category_ids)
image_data = example_coco.loadImgs(0)[0]

In [ ]:
image_data

In [ ]:
# load and display instance annotations
image = io.imread(path_to_image + image_data['file_name'])
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
annotation_ids = example_coco.getAnnIds(imgIds=image_data['id'])
annotations = example_coco.loadAnns(annotation_ids)
example_coco.showAnns(annotations, draw_bbox=True)

In [ ]:
image_data['id']

Annotation from Signate:
```
{"Car": [
            {"id": 5598, "box2d": [1825, 577, 1935, 671]}, 
            {"id": 5538, "box2d": [1181, 486, 1762, 897]}, 
            {"id": 5597, "box2d": [1099, 579, 1196, 713]}, 
            {"id": 5502, "box2d": [812, 583, 1088, 820]}, 
            {"id": 5596, "box2d": [1047, 552, 1158, 660]}
        ], 
        "Pedestrian": [
            {"id": 37925, "box2d": [620, 556, 647, 602]},
            {"id": 37923, "box2d": [588, 545, 611, 600]}
        ], 
        "Signal": [
            {"id": 181, "box2d": [1263, 453, 1304, 481]}
        ]}
        ```

Annotation from the COCO json generated:
```
[{"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [1825, 1935, 110, 94], "category_id": 1, "id": 5598}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [1181, 1762, 581, 411], "category_id": 1, "id": 5538}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [1099, 1196, 97, 134], "category_id": 1, "id": 5597}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [812, 1088, 276, 237], "category_id": 1, "id": 5502}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [1047, 1158, 111, 108], "category_id": 1, "id": 5596}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [620, 647, 27, 46], "category_id": 2, "id": 37925}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [588, 611, 23, 55], "category_id": 2, "id": 37923}, 
 {"segmentation": [], "area": 0, "iscrowd": 0, "image_id": 0, "bbox": [1263, 1304, 41, 28], "category_id": 4, "id": 181},
 ```

In [ ]:
lenght = len(annotation_ids)
print("Number of object in this frame {}, ids: {}".format(lenght, annotation_ids))

In [ ]:
annotations